# Similitud semántica

Vamos a ilustrar en qué consiste la similitud semántica con un pequeño ejemplo de buscador que utiliza embeddings.

Para ello, vamos a inspirarnos es este [ejemplo de buscador semántico en 3 minutos](https://github.com/hanxiao/bert-as-service#building-a-qa-semantic-search-engine-in-3-minutes), del proyecto [`bert-as-service`](https://github.com/hanxiao/bert-as-service) de [Han Xiao](https://hanxiao.io/), y simular que estamos construyendo un motor de búsqueda semántico sencillo.

In [ ]:
import numpy as np
import spacy

from typing import List

nlp = spacy.load("en_core_web_md")

Inventamos unos cuantos documentos (en este caso, simples oraciones) de ejemplo, con poco solapamiento entre ellas. Construimos los embeddings de las oraciones utilizando los vectores de GloVe incluidos en el modelo de inglés disponible con spaCy.

In [ ]:
docs = [
    "I hate tennis.",
    "I love ice cream",
    "I love riding my bicycle",
    "My car is a blue Hyunday",
    "This session is about natural language processing and artificial intelligence"
]

doc_vecs = [nlp(doc).vector for doc in docs]

Definimos una función que toma como argumentos de entrada un texto como query, el número de resultados que queremos mostrar, una colección de documentos en formato texto, y la misma colección de documentos vectorizada.

La función procesa la query de entrada con el modelo de spaCy, construye el embedding y calcula el producto escalar con respecto a todos y cada uno de los embeddings de la colección de documentos. Después, imprime los resultados más similares.

In [ ]:
def search_similar_query(
    text: str, 
    topk: int,
    docs: List[str], 
    doc_vecs: List[np.ndarray]
):
    """Search for similar vectors.
    It computes the dot product between a vector and a set of vectors"""
    query = nlp(text)
    # normalized dot product as score
    score = np.sum(query.vector * doc_vecs, axis=1) / np.linalg.norm(doc_vecs, axis=1)
    topk_idx = np.argsort(score)[::-1][:topk]
    for idx in topk_idx:
        print(f"[{score[idx]}]: {docs[idx]}")

In [ ]:
search_similar_query("I don't like racket sports.", topk=3, docs=docs, doc_vecs=doc_vecs)

In [ ]:
search_similar_query("I own a Toyota vehicle.", topk=3, docs=docs, doc_vecs=doc_vecs)

In [ ]:
search_similar_query("I enjoy desserts.", topk=3, docs=docs, doc_vecs=doc_vecs)

In [ ]:
search_similar_query("We are talking about deep learning.", topk=3, docs=docs, doc_vecs=doc_vecs)

In [ ]:
search_similar_query("I have too many bikes", topk=3, docs=docs, doc_vecs=doc_vecs)

## Corpus de Chatbots

A continuación vamos a repetir el proceso con una colección de mensajes más grande. En el directorio `data/` del repo he incluido un par de ficheros CSV que continene datos de entrenamiento para un chatbot en inglés y en español. Vamos a cargar los datos del dataset en inglés.